## 事件流处理器

In [1]:
from illufly.io import EventBlock
EventBlock("Chunk", "hi")

EventBlock(block_type=<chunk>, content=<hi>)

### 实现流交换

基于大模型的AI应用中经常要求模型流式返回，我们有一种实现流输出的标准实现。

In [1]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield "hi\n"
        yield "illufly!\n"

r = MyRun()
r()

hi
illufly!


In [2]:
r.events_history.store

{'20241129-69372-0000-9460': {'agents': {'MyRun.4406787952': {}},
  'callings': {'20241129-69372-0000-6976': [{'id': '20241129-69372-0000-2767',
     'event': 'message',
     'data': '{"block_id": "20241129-69372-0000-2767", "block_type": "runnable", "content": "MyRun.4406787952", "content_id": "20241129-69372-0001-3225", "created_at": "2024-11-29T16:36:12.210579", "thread_id": null, "calling_id": "20241129-69372-0000-6976", "agent_name": "MyRun.4406787952", "model_name": null}'},
    {'id': '20241129-69372-0002-3321',
     'event': 'message',
     'data': '{"block_id": "20241129-69372-0002-3321", "block_type": "text", "content": "hi\\n", "content_id": "20241129-69372-0003-6882", "created_at": "2024-11-29T16:36:12.210644", "thread_id": null, "calling_id": "20241129-69372-0000-6976", "agent_name": "MyRun.4406787952", "model_name": null}'},
    {'id': '20241129-69372-0004-2970',
     'event': 'message',
     'data': '{"block_id": "20241129-69372-0004-2970", "block_type": "text", "content

### 推送事件流

In [4]:
for x in r(generator="sync", verbose=False):
    print("SSE >>", x)

SSE >> {'id': '20241129-67723-0012-7849', 'event': 'message', 'data': '{"block_id": "20241129-67723-0012-7849", "block_type": "text", "content": "hi\\n", "content_id": "20241129-67723-0013-8387", "created_at": "2024-11-29T16:08:43.427174", "thread_id": null, "calling_id": "20241129-67723-0001-5231", "agent_name": "MyRun.4568773488", "model_name": null}'}
SSE >> {'id': '20241129-67723-0016-0648', 'event': 'message', 'data': '{"block_id": "20241129-67723-0016-0648", "block_type": "text", "content": "illufly!\\n", "content_id": "20241129-67723-0017-2515", "created_at": "2024-11-29T16:08:43.427482", "thread_id": null, "calling_id": "20241129-67723-0001-5231", "agent_name": "MyRun.4568773488", "model_name": null}'}


In [5]:
async for x in r(generator="async", verbose=False):
    print("SSE >>", x)

SSE >> {'id': '20241129-67724-0022-0058', 'event': 'message', 'data': '{"block_id": "20241129-67724-0022-0058", "block_type": "text", "content": "hi\\n", "content_id": "20241129-67724-0023-6530", "created_at": "2024-11-29T16:08:44.028100", "thread_id": null, "calling_id": "20241129-67724-0002-3152", "agent_name": "MyRun.4568773488", "model_name": null}'}
SSE >> {'id': '20241129-67724-0026-1436', 'event': 'message', 'data': '{"block_id": "20241129-67724-0026-1436", "block_type": "text", "content": "illufly!\\n", "content_id": "20241129-67724-0027-9011", "created_at": "2024-11-29T16:08:44.028664", "thread_id": null, "calling_id": "20241129-67724-0002-3152", "agent_name": "MyRun.4568773488", "model_name": null}'}


In [6]:
r.events_history.store

{'20241129-67663-0000-0376': {'threads': {},
  'callings': {'20241129-67663-0000-4660': [{'id': '20241129-67663-0002-3594',
     'event': 'message',
     'data': '{"block_id": "20241129-67663-0002-3594", "block_type": "runnable", "content": "MyRun.4568773488", "content_id": "20241129-67663-0003-9023", "created_at": "2024-11-29T16:07:43.601380", "thread_id": null, "calling_id": "20241129-67663-0000-4660", "agent_name": "MyRun.4568773488", "model_name": null}'},
    {'id': '20241129-67663-0004-9015',
     'event': 'message',
     'data': '{"block_id": "20241129-67663-0004-9015", "block_type": "text", "content": "hi\\n", "content_id": "20241129-67663-0005-9224", "created_at": "2024-11-29T16:07:43.601449", "thread_id": null, "calling_id": "20241129-67663-0000-4660", "agent_name": "MyRun.4568773488", "model_name": null}'},
    {'id': '20241129-67663-0006-5237',
     'event': 'message',
     'data': '{"block_id": "20241129-67663-0006-5237", "block_type": "text", "content": "illufly!\\n", "co

In [7]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield ["hi\n", "illufly!\n"]

r = MyRun()
r()

['hi\n', 'illufly!\n']

### Runnable 事件流

In [8]:
from illufly.types import Runnable, EventBlock

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield EventBlock("chunk", "hi, ")
        yield EventBlock("chunk", "illufly")
        yield EventBlock("chunk", "!")

def handle(block, **kwargs):
    print(block.json)

r = MyRun()
r("hi")

hi, illufly!

In [13]:
r.events_history.store

{'20241129-67758-0004-7641': {'threads': {'FakeLLM.4706565520': '20241129-67758-0001-7056'},
  'callings': {'20241129-67758-0006-8941': [{'id': '20241129-67758-0075-5306',
     'event': 'message',
     'data': '{"block_id": "20241129-67758-0075-5306", "block_type": "runnable", "content": "FakeLLM.4706565520", "content_id": "20241129-67758-0076-4983", "created_at": "2024-11-29T16:09:18.700119", "thread_id": null, "calling_id": "20241129-67758-0006-8941", "agent_name": "FakeLLM.4706565520", "model_name": null}'},
    {'id': '20241129-67758-0077-3636',
     'event': 'message',
     'data': '{"block_id": "20241129-67758-0077-3636", "block_type": "human", "content": "hi", "content_id": "20241129-67758-0078-1555", "created_at": "2024-11-29T16:09:18.701752", "thread_id": null, "calling_id": "20241129-67758-0006-8941", "agent_name": "FakeLLM.4706565520", "model_name": null}'},
    {'id': '20241129-67758-0079-1297',
     'event': 'message',
     'data': '{"block_id": "20241129-67758-0079-1297",

### ChatAgent 事件流

In [3]:
from illufly.types import Runnable, EventBlock
from illufly.chat import FakeLLM
from illufly.io import log, alog

def my_handler(block, **kwargs):
    print("[", block.block_type, "]", block.text)

r = FakeLLM(handlers=[my_handler], sleep=0.5)
r("hi")

[ runnable ] FakeLLM.4544084432
[ human ] hi
[ agent ] FakeLLM.4544084432
[ info ] 记住 10 轮对话
[ info ] I am FakeLLM
[ chunk ] R
[ chunk ] e
[ chunk ] p
[ chunk ] l
[ chunk ] y
[ chunk ]  
[ chunk ] >
[ chunk ] >
[ chunk ]  
[ chunk ] h
[ chunk ] i
[ new_line ] 
[ final_text ] Reply >> hi


'Reply >> hi'

In [11]:
from illufly.types import Runnable, EventBlock
from illufly.chat import FakeLLM
from illufly.io import log, alog

r = FakeLLM(handlers=[alog], sleep=0.1)
await r("hi")

[HUMAN] hi
[AGENT] FakeLLM.4706565520
Reply >> hi


'Reply >> hi'

In [4]:
r.events_history.store

{'20241129-69390-0001-7048': {'agents': {'FakeLLM.4544084432': {'thread_id': '20241129-69390-0000-2507'}},
  'callings': {'20241129-69390-0001-6224': [{'id': '20241129-69390-0006-6661',
     'event': 'message',
     'data': '{"block_id": "20241129-69390-0006-6661", "block_type": "runnable", "content": "FakeLLM.4544084432", "content_id": "20241129-69390-0007-4346", "created_at": "2024-11-29T16:36:30.200062", "thread_id": null, "calling_id": "20241129-69390-0001-6224", "agent_name": "FakeLLM.4544084432", "model_name": null}'},
    {'id': '20241129-69390-0008-1468',
     'event': 'message',
     'data': '{"block_id": "20241129-69390-0008-1468", "block_type": "human", "content": "hi", "content_id": "20241129-69390-0009-0773", "created_at": "2024-11-29T16:36:30.200737", "thread_id": null, "calling_id": "20241129-69390-0001-6224", "agent_name": "FakeLLM.4544084432", "model_name": null}'},
    {'id': '20241129-69390-0010-6906',
     'event': 'message',
     'data': '{"block_id": "20241129-693

### 推送 ChatAgent 事件流

In [14]:
from illufly.chat import FakeLLM

llm = FakeLLM(name="if1", handlers=[])
llm.create_new_thread()

async for b in llm("给我写一篇200字小作文", generator="async", sleep=0.1):
    print(b)

{'id': '20241129-68858-0114-1367', 'event': 'message', 'data': '{"block_id": "20241129-68858-0114-1367", "block_type": "agent", "content": "if1", "content_id": "20241129-68858-0115-6858", "created_at": "2024-11-29T16:27:38.055149", "thread_id": "20241129-68858-0002-4511", "calling_id": "20241129-68858-0007-8794", "agent_name": "if1", "model_name": null}'}
{'id': '20241129-68858-0116-0904', 'event': 'message', 'data': '{"block_id": "20241129-68858-0116-0904", "block_type": "info", "content": "记住 10 轮对话", "content_id": "20241129-68858-0117-6881", "created_at": "2024-11-29T16:27:38.055767", "thread_id": "20241129-68858-0002-4511", "calling_id": "20241129-68858-0007-8794", "agent_name": "if1", "model_name": null}'}
{'id': '20241129-68858-0118-2293', 'event': 'message', 'data': '{"block_id": "20241129-68858-0118-2293", "block_type": "info", "content": "I am FakeLLM", "content_id": "cac5e6e0ae2b11ef868eacde48001122", "created_at": "2024-11-29T16:27:38.058581", "thread_id": "20241129-68858-00

In [15]:
async for b in llm("给我写一篇200字小作文", generator="async", sleep=0.1):
    print(b)

{'id': '20241129-68864-0169-5704', 'event': 'message', 'data': '{"block_id": "20241129-68864-0169-5704", "block_type": "agent", "content": "if1", "content_id": "20241129-68864-0170-6255", "created_at": "2024-11-29T16:27:44.699536", "thread_id": "20241129-68858-0002-4511", "calling_id": "20241129-68864-0008-3721", "agent_name": "if1", "model_name": null}'}
{'id': '20241129-68864-0171-2734', 'event': 'message', 'data': '{"block_id": "20241129-68864-0171-2734", "block_type": "info", "content": "记住 10 轮对话", "content_id": "20241129-68864-0172-7353", "created_at": "2024-11-29T16:27:44.700576", "thread_id": "20241129-68858-0002-4511", "calling_id": "20241129-68864-0008-3721", "agent_name": "if1", "model_name": null}'}
{'id': '20241129-68864-0173-0623', 'event': 'message', 'data': '{"block_id": "20241129-68864-0173-0623", "block_type": "info", "content": "I am FakeLLM", "content_id": "cebbbaeaae2b11ef868eacde48001122", "created_at": "2024-11-29T16:27:44.702077", "thread_id": "20241129-68858-00

In [17]:
llm.events_history.store

{'20241129-68858-0005-6515': {'threads': {'if1': '20241129-68858-0002-4511'},
  'callings': {'20241129-68858-0007-8794': [{'id': '20241129-68858-0110-5098',
     'event': 'message',
     'data': '{"block_id": "20241129-68858-0110-5098", "block_type": "runnable", "content": "if1", "content_id": "20241129-68858-0111-3765", "created_at": "2024-11-29T16:27:38.054933", "thread_id": "20241129-68858-0002-4511", "calling_id": "20241129-68858-0007-8794", "agent_name": "if1", "model_name": null}'},
    {'id': '20241129-68858-0112-9540',
     'event': 'message',
     'data': '{"block_id": "20241129-68858-0112-9540", "block_type": "human", "content": "给我写一篇200字小作文", "content_id": "20241129-68858-0113-7593", "created_at": "2024-11-29T16:27:38.055086", "thread_id": "20241129-68858-0002-4511", "calling_id": "20241129-68858-0007-8794", "agent_name": "if1", "model_name": null}'},
    {'id': '20241129-68858-0114-1367',
     'event': 'message',
     'data': '{"block_id": "20241129-68858-0114-1367", "bloc

## 嵌套 BaseAgent

In [18]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen
from illufly.flow import ReAct

def get_city(location: str):
    """由任意地名或地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

flow = ReAct(
    ChatQwen(name="react-planner", tools=[get_city, get_weather, booking])
)

for b in flow("我在鸿蒙公司，帮我查询一下天气情况", verbose=False, generator="sync"):
    print(b)

{'id': '20241129-68954-0222-0604', 'event': 'message', 'data': '{"block_id": "20241129-68954-0222-0604", "block_type": "node", "content": ">>> flow-node-1: planner", "content_id": "20241129-68954-0223-5492", "created_at": "2024-11-29T16:29:14.720081", "thread_id": null, "calling_id": "20241129-68954-0009-2360", "agent_name": "ReAct.4715673200", "model_name": null}'}
{'id': '20241129-68954-0226-9602', 'event': 'message', 'data': '{"block_id": "20241129-68954-0226-9602", "block_type": "agent", "content": "react-planner", "content_id": "20241129-68954-0227-9505", "created_at": "2024-11-29T16:29:14.720433", "thread_id": null, "calling_id": "20241129-68954-0009-2360", "agent_name": "react-planner", "model_name": "qwen-plus"}'}
{'id': '20241129-68954-0228-5002', 'event': 'message', 'data': '{"block_id": "20241129-68954-0228-5002", "block_type": "info", "content": "记住 10 轮对话", "content_id": "20241129-68954-0229-2643", "created_at": "2024-11-29T16:29:14.720522", "thread_id": null, "calling_id"

{'20241129-68954-0012-9187': {'threads': {'react-planner': '20241129-68954-0003-9478'},
  'callings': {'20241129-68954-0009-2360': [{'id': '20241129-68954-0220-4626',
     'event': 'message',
     'data': '{"block_id": "20241129-68954-0220-4626", "block_type": "runnable", "content": "ReAct.4715673200", "content_id": "20241129-68954-0221-8732", "created_at": "2024-11-29T16:29:14.719924", "thread_id": null, "calling_id": "20241129-68954-0009-2360", "agent_name": "ReAct.4715673200", "model_name": null}'},
    {'id': '20241129-68954-0222-0604',
     'event': 'message',
     'data': '{"block_id": "20241129-68954-0222-0604", "block_type": "node", "content": ">>> flow-node-1: planner", "content_id": "20241129-68954-0223-5492", "created_at": "2024-11-29T16:29:14.720081", "thread_id": null, "calling_id": "20241129-68954-0009-2360", "agent_name": "ReAct.4715673200", "model_name": null}'},
    {'id': '20241129-68954-0224-4521',
     'event': 'message',
     'data': '{"block_id": "20241129-68954-0

In [19]:
flow.events_history.store

{'20241129-68954-0012-9187': {'threads': {'react-planner': '20241129-68954-0003-9478'},
  'callings': {'20241129-68954-0009-2360': [{'id': '20241129-68954-0220-4626',
     'event': 'message',
     'data': '{"block_id": "20241129-68954-0220-4626", "block_type": "runnable", "content": "ReAct.4715673200", "content_id": "20241129-68954-0221-8732", "created_at": "2024-11-29T16:29:14.719924", "thread_id": null, "calling_id": "20241129-68954-0009-2360", "agent_name": "ReAct.4715673200", "model_name": null}'},
    {'id': '20241129-68954-0222-0604',
     'event': 'message',
     'data': '{"block_id": "20241129-68954-0222-0604", "block_type": "node", "content": ">>> flow-node-1: planner", "content_id": "20241129-68954-0223-5492", "created_at": "2024-11-29T16:29:14.720081", "thread_id": null, "calling_id": "20241129-68954-0009-2360", "agent_name": "ReAct.4715673200", "model_name": null}'},
    {'id': '20241129-68954-0224-4521',
     'event': 'message',
     'data': '{"block_id": "20241129-68954-0

## Events History

In [1]:
from illufly.io import LocalFileEventsHistory
from illufly.chat import FakeLLM
llm = FakeLLM(events_history=LocalFileEventsHistory("__EVENTS__"))

In [3]:
llm.events_history.load_events_history()

('20241128-08897-0000-6863',
 {'threads': [['FakeLLM.4413207552', '20241128-08903-0000-3182'],
   ['FakeLLM.4519100032', '20241129-45516-0005-5280']],
  'callings': {'20241128-08903-0000-7527': {'id': '20241128-08903-0000-1506',
    'event_type': 'log',
    'data': '{"block_id": "20241128-08903-0000-1506", "block_type": "runnable", "content": "FakeLLM.4413207552", "content_id": "20241128-08903-0001-5465", "created_at": "2024-11-28T23:48:23.412970", "thread_id": null, "calling_id": "20241128-08903-0000-7527", "agent_name": "FakeLLM.4413207552", "model_name": null}'},
   '20241129-45516-0011-9642': {'id': '20241129-45516-0288-6749',
    'event_type': 'log',
    'data': '{"block_id": "20241129-45516-0288-6749", "block_type": "runnable", "content": "FakeLLM.4519100032", "content_id": "20241129-45516-0289-2091", "created_at": "2024-11-29T09:58:36.434904", "thread_id": null, "calling_id": "20241129-45516-0011-9642", "agent_name": "FakeLLM.4519100032", "model_name": null}'}}})

In [16]:
llm.events_history.events_history_id

'20241128-08897-0000-6863'

In [17]:
llm.events_history.store

{'20241128-08897-0000-6863': {'threads': [['FakeLLM.4413207552',
    '20241128-08903-0000-3182']],
  'callings': {'20241128-08903-0000-7527': {'id': '20241128-08903-0000-1506',
    'event_type': 'log',
    'data': '{"block_id": "20241128-08903-0000-1506", "block_type": "runnable", "content": "FakeLLM.4413207552", "content_id": "20241128-08903-0001-5465", "created_at": "2024-11-28T23:48:23.412970", "thread_id": null, "calling_id": "20241128-08903-0000-7527", "agent_name": "FakeLLM.4413207552", "model_name": null}'}}}}

In [18]:
llm("hello")

[AGENT] FakeLLM.4519100032
Reply >> hello


'Reply >> hello'

In [2]:
llm.events_history.load_events_history(-1)

('20241128-08897-0000-6863',
 {'threads': [['FakeLLM.4413207552', '20241128-08903-0000-3182'],
   ['FakeLLM.4519100032', '20241129-45516-0005-5280']],
  'callings': {'20241128-08903-0000-7527': {'id': '20241128-08903-0000-1506',
    'event_type': 'log',
    'data': '{"block_id": "20241128-08903-0000-1506", "block_type": "runnable", "content": "FakeLLM.4413207552", "content_id": "20241128-08903-0001-5465", "created_at": "2024-11-28T23:48:23.412970", "thread_id": null, "calling_id": "20241128-08903-0000-7527", "agent_name": "FakeLLM.4413207552", "model_name": null}'},
   '20241129-45516-0011-9642': {'id': '20241129-45516-0288-6749',
    'event_type': 'log',
    'data': '{"block_id": "20241129-45516-0288-6749", "block_type": "runnable", "content": "FakeLLM.4519100032", "content_id": "20241129-45516-0289-2091", "created_at": "2024-11-29T09:58:36.434904", "thread_id": null, "calling_id": "20241129-45516-0011-9642", "agent_name": "FakeLLM.4519100032", "model_name": null}'}}})